<a href="https://colab.research.google.com/github/FaridRash/Advance-Statistical-Learnin--Semester-3/blob/main/MH/GLM_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Libraries

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


#Git

In [ ]:
!git clone https://github.com/FaridRash/Advance-Statistical-Learnin--Semester-3

Cloning into 'Advance-Statistical-Learnin--Semester-3'...
remote: Enumerating objects: 541, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 541 (delta 108), reused 11 (delta 11), pack-reused 373 (from 2)
Receiving objects: 100% (541/541), 57.70 MiB | 11.34 MiB/s, done.
Resolving deltas: 100% (265/265), done.


#Data Reading

In [ ]:
df = pd.read_csv('/content/Advance-Statistical-Learnin--Semester-3/Dataset/df_Final.csv')
display(df.head())

,Patient_ID,Visit_ID,Sex,Age_at_onset,Clinical_onset_type,Spirometry,FVC,Ventilation,Tracheostomy,PEG,...,MRC_Bulbar,PUMNS_Upper_Limb,PUMNS_Lower_Limb,PUMNS_Bulbar,CNS_Linear_Score,KINGS_Total,Disease_duration,Progression_rate,ALSFRS_R,ALSAQ_5
0,133,3,0,19.0,spinale,1.0,>80,0.0,0,0.0,...,14.0,0.0,6.0,0.0,7.0,1,40.0,0.075000,45.0,0.0
1,79,1,0,65.0,spinale,1.0,40-60,0.0,0,0.0,...,13.0,1.0,4.0,3.0,15.0,1,14.0,0.714286,38.0,15.0
2,81,1,1,79.0,bulbare,1.0,>80,0.0,0,0.0,...,11.0,4.0,4.0,2.0,7.0,1,6.0,0.833333,43.0,20.0
3,214,1,0,51.0,spinale,1.0,>80,0.0,0,0.0,...,15.0,0.0,0.0,0.0,7.0,1,20.0,0.200000,44.0,20.0
4,214,2,0,51.0,spinale,1.0,>80,0.0,0,0.0,...,15.0,0.0,0.0,0.0,7.0,1,23.0,0.130435,45.0,25.0


In [ ]:
display(df.columns)

Index(['Patient_ID', 'Visit_ID', 'Sex', 'Age_at_onset', 'Clinical_onset_type',
       'Spirometry', 'FVC', 'Ventilation', 'Tracheostomy', 'PEG', 'Therapy',
       'Diagnostic_delay', 'MRC_Upper_Limb', 'MRC_Lower_Limb', 'MRC_Bulbar',
       'PUMNS_Upper_Limb', 'PUMNS_Lower_Limb', 'PUMNS_Bulbar',
       'CNS_Linear_Score', 'KINGS_Total', 'Disease_duration',
       'Progression_rate', 'ALSFRS_R', 'ALSAQ_5'],
      dtype='object')

#GLM

In [ ]:
# === Baseline GLM for KINGS_Total (ordinal logistic, cross-sectional) ===
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import accuracy_score, mean_absolute_error, confusion_matrix
from statsmodels.miscmodels.ordinal_model import OrderedModel

# 2) Target as ordered categorical (1 < 2 < 3 < 4)
df["KINGS_Total"] = pd.Categorical(df["KINGS_Total"],
                                   categories=[1, 2, 3, 4],
                                   ordered=True)

# 3) Pick a simple, clinically sensible first wave of predictors
predictors = ["Disease_duration", "Age_at_onset", "Sex", "Therapy", "FVC"]

# 4) Types: categoricals + numerics
for c in ["Sex", "Therapy", "FVC"]:
    if c in df.columns:
        df[c] = df[c].astype("category")
for c in ["Disease_duration", "Age_at_onset"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 5) Keep only what we need; drop missing
cols = ["Patient_ID", "KINGS_Total"] + predictors
d = df[cols].dropna().copy()

# 6) Build X (one-hot for categoricals) and y (0..3 codes), keep patient ids for grouped split
X = pd.get_dummies(d[predictors], drop_first=True).astype(float)
y = d["KINGS_Total"].cat.codes.astype(int).values   # 0,1,2,3
pid = d["Patient_ID"].values

# 7) Grouped train/test split (no leakage across patients)
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=pid))
Xtr, Xte = X.iloc[train_idx], X.iloc[test_idx]
ytr, yte = y[train_idx], y[test_idx]

# 8) Fit naïve GLM (proportional-odds ordinal logistic)
mod_glm = OrderedModel(ytr, Xtr, distr="logit")
res_glm = mod_glm.fit(method="lbfgs", maxiter=200, disp=False)

# 9) Inspect + quick evaluation
print("=== Naïve GLM (Ordinal Logistic) Summary ===")
print(res_glm.summary())

# Predicted class = argmax of predicted probabilities
probs = res_glm.predict(Xte)                       # shape (n_test, 4)
yhat  = np.asarray(probs).argmax(axis=1)

print("\n=== Test set metrics ===")
print("Accuracy:", round(accuracy_score(yte, yhat), 3))
print("Ordinal MAE (stage distance):", round(mean_absolute_error(yte, yhat), 3))
print("\nConfusion matrix (rows=true, cols=pred; stages coded 0..3):\n",
      confusion_matrix(yte, yhat, labels=[0,1,2,3]))

=== Naïve GLM (Ordinal Logistic) Summary ===
                             OrderedModel Results                             
Dep. Variable:                      y   Log-Likelihood:                -1383.5
Model:                   OrderedModel   AIC:                             2791.
Method:            Maximum Likelihood   BIC:                             2852.
Date:                Tue, 02 Sep 2025                                         
Time:                        11:59:00                                         
No. Observations:                1208                                         
Df Residuals:                    1196                                         
Df Model:                           9                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Disease_duration                 0.0139      0.002

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import accuracy_score, mean_absolute_error, confusion_matrix
from statsmodels.miscmodels.ordinal_model import OrderedModel

# --- 1) load + target ordering
df["KINGS_Total"] = pd.Categorical(df["KINGS_Total"], categories=[1,2,3,4], ordered=True)

# --- 2) types
cat_cols = ["Sex", "Therapy", "FVC"]
for c in cat_cols:
    if c in df.columns:
        df[c] = df[c].astype("category")

# (optional but helpful) set "nessuna terapia" as baseline if present
if "Therapy" in df.columns and "nessuna terapia" in list(df["Therapy"].cat.categories):
    cats = list(df["Therapy"].cat.categories)
    cats = ["nessuna terapia"] + [x for x in cats if x != "nessuna terapia"]
    df["Therapy"] = df["Therapy"].cat.reorder_categories(cats)

num_cols = [
    "Disease_duration", "Age_at_onset",
    "MRC_Upper_Limb", "MRC_Lower_Limb", "MRC_Bulbar",
    "PUMNS_Upper_Limb", "PUMNS_Lower_Limb", "PUMNS_Bulbar",
    "CNS_Linear_Score"
]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# --- 3) modeling frame
predictors = ["Disease_duration","Age_at_onset","Sex","Therapy","FVC",
              "MRC_Upper_Limb","MRC_Lower_Limb","MRC_Bulbar",
              "PUMNS_Upper_Limb","PUMNS_Lower_Limb","PUMNS_Bulbar",
              "CNS_Linear_Score"]

cols = ["Patient_ID","KINGS_Total"] + [c for c in predictors if c in df.columns]
d = df[cols].dropna().copy()

# --- 4) design matrix: dummies for categoricals, numeric for the rest
X = pd.get_dummies(d[predictors], drop_first=True).astype(float)
y = d["KINGS_Total"].cat.codes.astype(int).values
pid = d["Patient_ID"].values

# --- 5) grouped train/test split (no patient leakage)
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=pid))
Xtr, Xte = X.iloc[train_idx], X.iloc[test_idx]
ytr, yte = y[train_idx], y[test_idx]

# --- 6) fit GLM (ordinal logistic)
mod_glm = OrderedModel(ytr, Xtr, distr="logit")
res_glm = mod_glm.fit(method="lbfgs", maxiter=400, disp=False)

print(res_glm.summary())

# --- 7) quick eval
probs = res_glm.predict(Xte)
yhat = np.asarray(probs).argmax(axis=1)
print("\nAccuracy:", round(accuracy_score(yte, yhat), 3))
print("Ordinal MAE:", round(mean_absolute_error(yte, yhat), 3))
print("Confusion matrix:\n", confusion_matrix(yte, yhat, labels=[0,1,2,3]))

                             OrderedModel Results                             
Dep. Variable:                      y   Log-Likelihood:                -1321.6
Model:                   OrderedModel   AIC:                             2681.
Method:            Maximum Likelihood   BIC:                             2778.
Date:                Tue, 02 Sep 2025                                         
Time:                        12:24:29                                         
No. Observations:                1208                                         
Df Residuals:                    1189                                         
Df Model:                          16                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Disease_duration                 0.0136      0.002      7.119      0.000       0.010       0.01

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [ ]:
# rebuild X so we can add interactions cleanly
X = pd.get_dummies(d[predictors], drop_first=True).astype(float)

# interaction: duration × each therapy dummy
ther_cols = [c for c in X.columns if c.startswith("Therapy_")]
for c in ther_cols:
    X[f"Dur_x_{c}"] = d["Disease_duration"].values * X[c].values

# optional: Sex × Age_at_onset
if "Sex_1" in X.columns:
    X["Age_x_Sex1"] = d["Age_at_onset"].values * X["Sex_1"].values

# split and refit
train_idx, test_idx = next(gss.split(X, y, groups=pid))
Xtr, Xte = X.iloc[train_idx], X.iloc[test_idx]
ytr, yte = y[train_idx], y[test_idx]

mod_glm = OrderedModel(ytr, Xtr, distr="logit")
res_glm = mod_glm.fit(method="lbfgs", maxiter=500, disp=False)

print(res_glm.summary())

probs = res_glm.predict(Xte)
yhat = np.asarray(probs).argmax(axis=1)
print("\nAccuracy:", round(accuracy_score(yte, yhat), 3))
print("Ordinal MAE:", round(mean_absolute_error(yte, yhat), 3))

                             OrderedModel Results                             
Dep. Variable:                      y   Log-Likelihood:                -1320.9
Model:                   OrderedModel   AIC:                             2688.
Method:            Maximum Likelihood   BIC:                             2805.
Date:                Tue, 02 Sep 2025                                         
Time:                        12:27:46                                         
No. Observations:                1208                                         
Df Residuals:                    1185                                         
Df Model:                          20                                         
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Disease_duration                       0.0144      0.003      4.801      0.000     

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
